In [26]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import numpy as np
import time
import math
import matplotlib.pyplot as plt
from numpy import interp
import fileinput
import sympy as sp
import struct
import random
from tqdm import tqdm
import numbers

#Processor=Overlay("./SPIKE_PROCESSOR/SPIKE_PROCESSOR.bit")
Processor=Overlay("./SNN_PROCESSOR/bas_belasi.bit")

In [27]:
CONTROLS_BASEADDR        = 0x43C00000
PARAMSPACE_BASEADDR      = 0x43C10000
SYNAPSES_BASEADDR        = 0x43C20000
SPIKE_OUT_DMA_BASEADDR   = 0x40400000
SPIKE_IN_DMA_BASEADDR    = 0x40410000
INTERRUPT_CONTROLLER     = 0x41800000
ADDRESS_RANGE            = 0xFFFF

# CONTROL REGISTERS

TOTAL_PARAMETER_MEMORY =  0
TOTAL_SYNAPTIC_MEMORY =  4
CROSSBAR_DIMENSIONS = 8
MAX_NEURONS= 12
PROCESSOR_CONTROLS = 16
NMC_DATA_ADDR = 20
ULEARN_ADDR_DATA = 24
XNEVER_HIGH_LOW = 28
GLOBAL_TIMESTEP_UPDATE_REQUEST = 32

# SYNAPSE INTERFACE

SYNAPSE_MEM_ADDR = 0
SYNAPSE_MEM_DIN = 4
SYNAPSE_MEM_DOUT = 8
SYNAPSE_MEM_EN = 12
SYNAPSE_MEM_WREN = 16
SYNAPSE_MEM_SELECT = 20

# PARAMETER INTERFACE

PARAMETER_MEM_ADDR = 0
PARAMETER_MEM_DIN = 4
PARAMETER_MEM_DOUT = 8
PARAMETER_MEM_EN = 12
PARAMETER_MEM_WREN = 16
PARAMETER_MEM_SELECT = 20

Controls = MMIO(CONTROLS_BASEADDR,ADDRESS_RANGE)
Parameter = MMIO(PARAMSPACE_BASEADDR,ADDRESS_RANGE)
Synapses = MMIO(SYNAPSES_BASEADDR,ADDRESS_RANGE)

In [28]:
SpikeOutStream = Processor.SPIKE_OUT_DMA 
SpikeInStream = Processor.SPIKE_IN_DMA 
PushSpikes = SpikeInStream.sendchannel
PullSpikes = SpikeOutStream.recvchannel

In [29]:
# Neuron Model Assembly code to NMC executable code.

def NModelAssembler(InputStringArray, ArrayBaseAddress):

    def lw_binary_from_string(command):
        parts = command.split(',')  
        reg_num = int(parts[1][1])  
        num = int(parts[2]) 
        reg_bin = format(reg_num, '03b')  
        num_bin = format(num, '09b') 
        binary = f"0001{reg_bin}{num_bin}" 
        return binary[-16:] 

    def sw_binary_from_string(command):
        parts = command.split(',')  
        reg_num = int(parts[1][1])  
        num = int(parts[2]) 
        reg_bin = format(reg_num, '03b')  
        num_bin = format(num, '09b')  
        binary = f"0010{reg_bin}{num_bin}"  
        return binary[-16:]  

    def fmac_binary_from_string(command):
        parts = command.split(',') 
        reg1_num = int(parts[1][1]) 
        reg2_num = int(parts[2][1]) 
        reg1_bin = format(reg1_num, '03b') 
        reg2_bin = format(reg2_num, '03b') 
        binary = f"0100{'000000'}{reg1_bin}{reg2_bin}"  
        return binary

    def smac_binary_from_string(command):
        parts = command.split(',')  
        reg1_num = int(parts[1][1]) 
        reg2_num = int(parts[2][1]) 
        reg1_bin = format(reg1_num, '03b')  
        reg2_bin = format(reg2_num, '03b')  
        binary = f"0101{'000000'}{reg1_bin}{reg2_bin}"  
        return binary

    def clracc_binary_from_string(command):
        binary = "0110000000000000"  
        return binary

    def comp_binary_from_string(command):
        parts = command.split(',') 
        reg1_num = int(parts[1][1])  
        reg2_num = int(parts[2][1])  
        reg1_bin = format(reg1_num, '03b')  
        reg2_bin = format(reg2_num, '03b') 
        binary = f"0111{'000000'}{reg1_bin}{reg2_bin}" 
        return binary

    def spk_binary_from_string(command):
        binary = "1011000000000000" 
        return binary

    def getacc_binary_from_string(command):
        parts = command.split(',')  
        register_num = int(parts[1][1])  
        reg_bin = format(register_num, '03b')  
        binary = f"0011{reg_bin}000000000" 
        return binary

    def return_binary_from_string(command):
        binary = "1101000000000000" 
        return binary

    def strf_binary_from_string(command):
        parts = command.split(',')  
        num = int(parts[1])  
        num_bin = format(num, '08b')  
        binary = f"11100000{num_bin}" 
        return binary

    def bil_binary(num):
        num_bin = format(num, '010b')  
        binary = f"1000{'00'}{num_bin}"  
        return binary

    def big_binary(num):
        num_bin = format(num, '010b')  
        binary = f"1010{'00'}{num_bin}"  
        return binary

    def bie_binary(num):
        num_bin = format(num, '010b')  
        binary = f"1001{'00'}{num_bin}"  
        return binary

    def convert_to_binary(command):
        if command.startswith('lw'):
            return lw_binary_from_string(command)
        elif command.startswith('sw'):
            return sw_binary_from_string(command)
        elif command.startswith('fmac'):
            return fmac_binary_from_string(command)
        elif command.startswith('smac'):
            return smac_binary_from_string(command)
        elif command.startswith('clr'):
            return clracc_binary_from_string(command)
        elif command.startswith('com'):
            return comp_binary_from_string(command)
        elif command.startswith('spk'):
            return spk_binary_from_string(command)
        elif command.startswith('get'):
            return getacc_binary_from_string(command)
        elif command.startswith('ret'):
            return return_binary_from_string(command)
        elif command.startswith('str'):
            return strf_binary_from_string(command)
        elif command.startswith('bil'):
            num = int(command[4:])
            return bil_binary(num)
        elif command.startswith('big'):
            num = int(command[4:])
            return big_binary(num)
        elif command.startswith('bie'):
            num = int(command[4:])
            return bie_binary(num)
        else:
            return "Invalid command"
        
    commands = InputStringArray.split('\n')
    NMC_Executable = []

    for command in commands:
        if command.strip():
            binary_representation = convert_to_binary(command)
            NMC_Executable.append(binary_representation)

    NMC_Binary = np.array([int(binary, 2) for binary in NMC_Executable], dtype=np.uint16)

    NMC_INT = NMC_Binary.astype(int)
    NMC_PROGRAM_DATA = []

    for i in range(len(NMC_INT)):
        NMC_PROGRAM_DATA.append(NMC_INT[i].item())

    SRM_ADDR = range(ArrayBaseAddress, ArrayBaseAddress + len(NMC_PROGRAM_DATA))
    SRM_ADDR_UINT32 = np.array(SRM_ADDR, dtype=int)

    SRM_ADDRESS_SPACE = []
    for i in range(len(SRM_ADDR_UINT32)):
        SRM_ADDRESS_SPACE.append(SRM_ADDR_UINT32[i].item())

    NMC_PROGRAM_DATA = []

    for i in range(len(SRM_ADDRESS_SPACE)):
        NMC_PROGRAM_DATA.append(NMC_INT[i].item() << 16 | SRM_ADDRESS_SPACE[i])

    return NMC_PROGRAM_DATA

In [30]:
def ULEARN_FIT(TimewindowLeft, TimewindowRight, ZeroPoint, Samples, expr1, expr2):
    t = sp.symbols('t')
    equation1 = sp.sympify(expr1)
    equation2 = sp.sympify(expr2)

    leftHandSpace = np.linspace(-TimewindowLeft, ZeroPoint, int(Samples/2))
    rightHandSpace = np.linspace(ZeroPoint, TimewindowRight, int(Samples/2))
    synapticUpdateFunctionLHS = np.zeros(int(Samples/2))
    synapticUpdateFunctionRHS = np.zeros(int(Samples/2))

    for i in range(0, len(leftHandSpace)):
        synapticUpdateFunctionLHS[i] = equation1.subs(t, leftHandSpace[i])
    
    for i in range(0, len(rightHandSpace)):
        synapticUpdateFunctionRHS[i] = equation2.subs(t, rightHandSpace[i])

    synapticFunction = np.concatenate((synapticUpdateFunctionLHS, synapticUpdateFunctionRHS), axis=None)
    timespace = np.concatenate((leftHandSpace, rightHandSpace), axis=None)

    qFactor = 2 / math.pow(2, 8)
    qSynapticFunction = synapticFunction / qFactor
    mSynapticFunction = qSynapticFunction.astype(int)
    qTimeWindow = np.linspace(-int(Samples/2), int(Samples/2) - 1, int(Samples))

    result = mSynapticFunction
    
    result_array=result.astype(np.int8)
    
    return result_array

# Create ULEARN Table

def LearningWindowMapper(LearningWindow):
    OnPreAddressSpace = np.arange(-128, 1, 1, dtype=np.int8)
    OnPostAddressSpace = np.arange(1, 128, dtype=np.int8)
    CompleteAddressSpace = np.concatenate((OnPreAddressSpace, OnPostAddressSpace))

    combined_binary = [bin(x & 0xFF)[2:].zfill(8) + bin(y & 0xFF)[2:].zfill(8) for x, y in zip(LearningWindow, CompleteAddressSpace)]

    ULEARN_REG = []
    ULEARN_REG_UINT16 = np.array([np.uint16(int(b, 2)) for b in combined_binary], dtype=np.uint16)

    for i in range(len(ULEARN_REG_UINT16)):
        ULEARN_REG.append(ULEARN_REG_UINT16[i].astype(int))

    ULEARN_DATA = np.array(ULEARN_REG, dtype=int)
    ULEARN_MAP = []

    for i in range(len(ULEARN_DATA)):
        ULEARN_MAP.append(ULEARN_DATA[i].item())

    return ULEARN_MAP

def Synapse(TimewindowLeft, TimewindowRight, ZeroPoint, Samples, expr1, expr2):
    SModel = ULEARN_FIT(TimewindowLeft, TimewindowRight, ZeroPoint, Samples, expr1, expr2)
    SModelTable = LearningWindowMapper(SModel)
    return SModelTable


def CreateRandomSynapse(length):
    return np.random.randint(0, 255, size=length, dtype=np.uint8)

def InitializeTimestamps(array):
    temp = array.astype(np.uint16)
    temp = temp << 8
    temp = temp | 0xFF
    return temp 

def Synapse2Stream(*arrays):
    uint64_arrays = [array.astype(np.uint64) for array in arrays]

    shifted_arrays = [np.left_shift(array, shift) for array, shift in zip(uint64_arrays, (48, 32, 16, 0))]

    result = np.bitwise_or.reduce(shifted_arrays)

    return result

def Stream2Synapse(result):
    shifts = (48, 32, 16, 0)
    
    reverse_shifted_arrays = [np.right_shift(result, shift) for shift in shifts]
    
    uint16_arrays = [array.astype(np.uint16) for array in reverse_shifted_arrays]
    
    return tuple(uint16_arrays)

def count_lines(file_path):
    try:
        with fileinput.input(files=(file_path,)) as file:
            line_count = sum(1 for _ in file)
        return line_count
    except FileNotFoundError:
        return 0

def ParameterAddressMerger(NPARAM,NADDR):
    MergedData = (NPARAM << 16) | NADDR
    return MergedData

def Spike2Pattern(uint16_array):
    bit_vectors = [np.unpackbits(np.array([num], dtype=np.uint16).view(np.uint8)) for num in uint16_array]
    bit_vector = np.concatenate([np.concatenate((bv[8:], bv[:8])) for bv in bit_vectors])
    bit_vector = bit_vector.reshape(-1, 1)
    return bit_vector

def reverse_bits(uint16_value):
    bit_array = np.unpackbits(np.array([uint16_value], dtype=np.uint16).view(np.uint8))
    
    reversed_bit_array = bit_array[::-1]
    
    reversed_uint16_value = np.packbits(reversed_bit_array).view(np.uint16)
    
    return reversed_uint16_value[0]

def signed_to_uint8(signed_val):
    if signed_val < 0:
        unsigned_val = signed_val + 256
    else:
        unsigned_val = signed_val
    return np.uint8(unsigned_val)

In [31]:
def HalfPrecision2Bin(float_num):
    single_precision = struct.pack('>f', float_num)
    single_as_int = struct.unpack('>I', single_precision)[0] 
    sign = (single_as_int >> 31) & 0x1
    exp = (single_as_int >> 23) & 0xFF
    mantissa = single_as_int & 0x7FFFFF
    new_exp = max(0, min(31, (exp - 127 + 15)))
    new_mantissa = mantissa >> 13
    half_precision = (sign << 15) | (new_exp << 10) | new_mantissa
    return half_precision


def Bin2HalfPrecision(int_val):
    sign = (int_val >> 15) & 0x1
    exp = (int_val >> 10) & 0x1F
    mantissa = int_val & 0x3FF
    if exp == 0:
        new_exp = 0
    elif exp == 31:
        new_exp = 255
    else:
        new_exp = exp - 15 + 127 
    new_mantissa = mantissa << 13
    single_precision = (sign << 31) | (new_exp << 23) | new_mantissa
    float_val = struct.unpack('>f', struct.pack('>I', single_precision))[0]
    return float_val

def FindMemAddrBitLength(n):
    if n < 1:
        raise ValueError("Address depth must be a positive integer.")
    exponent = math.ceil(math.log2(n))    
    power_of_2 = 2 ** exponent   
    return exponent

In [32]:

def ParseDataset(dataset, labels):
    unique_labels = np.unique(labels)  
    
    parsed_dataset = []
    parsed_labels = []
    
    for label in unique_labels:
        indices = np.where(labels == label)[0]  
        
        subset_dataset = dataset[indices]
        
        parsed_dataset.append(subset_dataset)
        parsed_labels.extend([label] * len(indices))
    
    return np.array(parsed_dataset), np.array(parsed_labels)

In [33]:
# Read Converted SEMION Spikes
# Read Packed Image Data

def DatasetLine2SpikeArray(filename, line_number):
    try:
        with open(filename, 'r') as file:
            for _ in range(line_number - 1):
                file.readline()
            line = file.readline().strip().split()
            uint16_array = np.array([int(value) for value in line], dtype=np.uint16)
            return uint16_array
    except FileNotFoundError:
        print(f"{filename} file not found.")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None



In [34]:
def ExtractSynapses(array_uint16):
    shape = array_uint16.shape
    array_uint8 = np.zeros(shape, dtype=np.uint8)
    for index, value in np.ndenumerate(array_uint16):
        extracted_value = np.uint8(value >> 8) 
        array_uint8[index] = extracted_value 
    return array_uint8

def ExtractTimestamps(array_uint16):
    shape = array_uint16.shape
    array_uint8 = np.zeros(shape, dtype=np.uint8)
    for index, value in np.ndenumerate(array_uint16):
        extracted_value = np.uint8(value & 0xFF) 
        array_uint8[index] = extracted_value
    return array_uint8

In [35]:
class Hardware:
    def __init__(self):
        self.MemoryNMC = int(1023)
        self.CrossbarColumns = None
        self.CrossbarRows = None
        self.SynapticMemoryPerColumn = None
        self.ParameterMemoryPerColumn = None
        self.NeuronCapacity = None
        self.FreeSynapticMemory = None
        self.FreeParameterMemory = None
        self.FreeNMCMemory = int(1023)
        
        self.LastSynapseAddress = None
        self.LastParameterAddress = None

        self.NMCXNeverLow = None
        self.NMCXNeverHigh = None
        
        self.SynapticMemoryAddressWidth = None
        self.ParameterMemoryAddressWidth = None


    def Generate(self):
        self.CrossbarColumns = Controls.read(CROSSBAR_DIMENSIONS)
        Row = (self.CrossbarColumns >> 16) & 0xFFFF
        Column = self.CrossbarColumns & 0xFFFF
        self.CrossbarRows= Row
        self.CrossbarColumns = Column
        TotalParamMemory = Controls.read(TOTAL_PARAMETER_MEMORY)
        TotalSynapticMemory = Controls.read(TOTAL_SYNAPTIC_MEMORY)        
        self.SynapticMemoryPerColumn = TotalSynapticMemory / self.CrossbarColumns
        self.ParameterMemoryPerColumn = TotalParamMemory / self.CrossbarColumns
        self.FreeSynapticMemory = [self.SynapticMemoryPerColumn] * self.CrossbarColumns
        self.FreeParameterMemory = [self.ParameterMemoryPerColumn] * self.CrossbarColumns
        self.LastSynapseAddress = [0] * self.CrossbarColumns
        self.LastParameterAddress = [0] * self.CrossbarColumns
        self.NeuronCapacity = Controls.read(MAX_NEURONS)     
        self.SynapticMemoryAddressWidth = FindMemAddrBitLength(self.SynapticMemoryPerColumn)
        self.ParameterMemoryAddressWidth = FindMemAddrBitLength(self.ParameterMemoryPerColumn)
        
    def GetHardwareInfo(self):     
        print(f"Crossbar dimensions {self.CrossbarRows}x{self.CrossbarColumns}.")
        print(f"{self.CrossbarColumns} Neuron Model Calculator (NMC) units.")
        print(f"{self.CrossbarColumns} Learning Engines.")
        print(f"Maximum {int(self.ParameterMemoryPerColumn*self.CrossbarColumns/8)} neurons.")
        print(f"Maximum {int(self.SynapticMemoryPerColumn*self.CrossbarColumns)} synapses.")
    
    def ShowFreeMemory(self): 
        for i in range(self.CrossbarColumns):
            synaptic_memory = int(self.FreeSynapticMemory[i])
            parameter_memory = int(self.FreeParameterMemory[i])
            print(f"Column {i}: Free Synaptic Memory {synaptic_memory} Words Free Parameter Memory {parameter_memory} Words")

    def SetExecuteNeverRegion(self, XNeverBase, XNeverHigh):
        try:
            start = int(XNeverBase)
            end = int(XNeverHigh)
            if isinstance(XNeverBase, float) or isinstance(XNeverHigh, float):
                raise ValueError("Only integers are allowed for setting NMC memory boundaries.")
            if start < 0 or start > self.MemoryNMC or end < 0 or end > self.MemoryNMC:
                raise ValueError("Execute never region boundaries cannot exceed NMC memory capacity which is fixed to 1023 words.")     
            self.NMCXNeverLow = start
            self.NMCXNeverHigh = end
            print(f"The XN region boundaries of all NMC units have been successfully set to {start} to {end}.") 
        except ValueError as e:
            raise ValueError(e) 
        HIGH_LOW = (end << 16) | start
        Controls.write(XNEVER_HIGH_LOW,HIGH_LOW)

    def LoadNeuronKernel(self,NeuronKernel):
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS);
        mask = (1 << 1) | (1 << 4) | (1 << 5)
        ControlRegTemp |= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        for i in range(len(NeuronKernel)):
            Controls.write(NMC_DATA_ADDR,NeuronKernel[i]) 
            self.FreeNMCMemory = self.FreeNMCMemory -1
        mask = ~((1 << 1) | (1 << 4) | (1 << 5))
        ControlRegTemp &= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        Controls.write(NMC_DATA_ADDR,0)  
    
    def LoadNeuronParameters(self,Data,Address): 
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS);
        mask = (1 << 1) | (1 << 4) | (1 << 5)
        ControlRegTemp |= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        if isinstance(Data, (list, tuple)):
            for DatIndx in range (len(Data)):
                NMCData = ParameterAddressMerger(Data[DatIndx], Address[DatIndx])
                Controls.write(NMC_DATA_ADDR,NMCData) 
                self.FreeNMCMemory = self.FreeNMCMemory -1
        else:
            NMCData = ParameterAddressMerger(Data,Address)
            Controls.write(NMC_DATA_ADDR,NMCData)
            self.FreeNMCMemory = self.FreeNMCMemory -1
            
        mask = ~((1 << 1) | (1 << 4) | (1 << 5))
        ControlRegTemp &= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        Controls.write(NMC_DATA_ADDR,0)  

    def LoadLearningTable(self,LearningTable):
        
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS);
        mask = (1 << 6)
        ControlRegTemp |= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
 
        if isinstance(LearningTable, (list, tuple)):
    
            for i in range(0,len(LearningTable)):
                Controls.write(ULEARN_ADDR_DATA,LearningTable[i])  
        else:
        
            Controls.write(ULEARN_ADDR_DATA,LearningTable)  
        mask = ~(1 << 6)
        ControlRegTemp &= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        Controls.write(ULEARN_ADDR_DATA,0)  

        
##########################################################################################################################                
    
    def MapNetwork(self, NeuralNet):
        NeuronIndexOffset = 0
        
        LayerIsInput = False
        LayerIsOutput = False
        LayerIsHidden = False
        ReadFromPropogator0 = False
        ReadFromPropogator1 = False
        WriteToPropogator0 = False
        WriteToPropogator1 = False
        ReadFromMain = False
        WriteToOut = False
        
        for LayerIndex, LayerSize in enumerate(NeuralNet.LayerSizes[1:], start=1): 
            
            if not NeuralNet.neurons:
                raise ValueError("NeuralNet.neurons is empty or None.")
            
            if LayerIndex == 1: 
                print(f"Mapping Input Layer with {LayerSize} neurons:")
                LayerIsInput  = True
                LayerIsOutput = False
                LayerIsHidden = False
            elif LayerIndex == len(NeuralNet.LayerSizes) - 1:  
                print(f"Mapping Output Layer with {LayerSize} neurons:")
                LayerIsInput  = False
                LayerIsOutput = True
                LayerIsHidden = False
            else: 
                print(f"Mapping Hidden Layer {LayerIndex - 1} with {LayerSize} neurons:")
                LayerIsInput  = False
                LayerIsOutput = False
                LayerIsHidden = True
                
            for NeuronInProcess in range(LayerSize):
                print(f"Processing Neuron ID = {NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].NeuronID}")  
                print(f"Number of Neuron Synapses = {len(NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].Synapses)}")
                neuron = NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset]
             
                if neuron is None:
                    print(f"Neuron at index {NeuronInProcess + NeuronIndexOffset} is None.")
                    continue 
            
                neuron_id = neuron.NeuronID
                column = (NeuronInProcess + NeuronIndexOffset) % self.CrossbarColumns
                print(f"Neuron ID {neuron_id} is being written to column {column}")
        
                if neuron.Synapses is None:
                    print(f"Neuron {neuron_id} has no synapses.")
                    continue 
                
                NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseLowAddress = self.LastSynapseAddress[column]
                NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].Position = column
                
                for SynapseValue in range(len(NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].Synapses)):
                    self.WriteNeuronSynapse(column,self.LastSynapseAddress[column],
                                            NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].Synapses[SynapseValue])
                    NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseHighAddress = self.LastSynapseAddress[column]
                    self.LastSynapseAddress[column]=self.LastSynapseAddress[column]+1
                    
                print(f"Neuron Synapse Low Address = {NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseLowAddress}")    
                print(f"Neuron Synapse High Address = {NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseHighAddress}")    
                
                print("Compiling Bridge Instructions for Neuron ID {neuron_id}")
                
########################################################## SYNLOW Command Generation #############################
                
                SYNLOW = 0b00010000000000000000000000000000

                synlowmask = (1 << self.SynapticMemoryAddressWidth) - 1  
                synapse_bits = NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseLowAddress & synlowmask  
    
                SYNLOW |= synapse_bits  
                print(f"SYNLOW Command = {bin(SYNLOW)}")
                print(f"Writing SYNLOW Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                self.WriteNeuronParameter(column,self.LastParameterAddress[column],SYNLOW)
                print(f"Reading back SYNLOW Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                SYNLOWBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                
                if SYNLOWBACK == SYNLOW:
                    print("Verified. Proceeding next.")
                else:
                    print("WRITE ERROR")
                
                self.LastParameterAddress[column] = self.LastParameterAddress[column]+1
                
########################################################## SSSDSYNHIGH Command Generation #############################
                
                SSSDSYNHIGH  = 0b00100000000000000000000000000000

                if (LayerIsInput == True and LayerIsHidden == False and LayerIsOutput == False):

                    ReadFromMain        = True
                    ReadFromPropogator0 = False
                    ReadFromPropogator1 = False
                    WriteToPropogator0  = True
                    WriteToPropogator1  = False
                    WriteToOut          = False
                    
                elif (LayerIsOutput == True and LayerIsInput == False and LayerIsHidden == False):
                    
                    if(WriteToPropogator0 == True and WriteToPropogator1 == False):
                        
                        ReadFromMain        = False
                        ReadFromPropogator0 = True
                        ReadFromPropogator1 = False
                        WriteToPropogator0  = False
                        WriteToPropogator1  = False
                        WriteToOut          = True

                    elif(WriteToPropogator0 == False and WriteToPropogator1 == True):
                        
                        ReadFromMain        = False
                        ReadFromPropogator0 = False
                        ReadFromPropogator1 = True
                        WriteToPropogator0  = False
                        WriteToPropogator1  = False
                        WriteToOut          = True
  
                elif (LayerIsHidden == True and LayerIsInput == False and LayerIsOutput == False):
                    
                    if(WriteToPropogator0 == True and WriteToPropogator1 == False):
                    
                        ReadFromMain        = False
                        ReadFromPropogator0 = True
                        ReadFromPropogator1 = False
                        WriteToPropogator0  = False
                        WriteToPropogator1  = True
                        WriteToOut          = False
                            
                    elif(WriteToPropogator0 == False and WriteToPropogator1 == True):
                        
                        ReadFromMain        = False
                        ReadFromPropogator0 = False
                        ReadFromPropogator1 = True
                        WriteToPropogator0  = True
                        WriteToPropogator1  = False
                        WriteToOut          = False
                            
                
                if(ReadFromMain == True and ReadFromPropogator0 == False and ReadFromPropogator1 == False 
                    and WriteToPropogator0 == True and WriteToPropogator1 == False and WriteToOut == False):
                    
                    SSSDSYNHIGH  = 0b00100000000010010000000000000000
                    
                elif(ReadFromMain == False and ReadFromPropogator0 == True and ReadFromPropogator1 == False 
                   and WriteToPropogator0 == False and WriteToPropogator1 == False and WriteToOut == True):
                    
                    SSSDSYNHIGH  = 0b00100000000100100000000000000000
                                                                   
                elif(ReadFromMain == False and ReadFromPropogator0 == False and ReadFromPropogator1 == True 
                        and WriteToPropogator0 == False and WriteToPropogator1 == False and WriteToOut == True):
                    
                    SSSDSYNHIGH  = 0b00100000000101000000000000000000
               
                elif(ReadFromMain == False and ReadFromPropogator0 == True and ReadFromPropogator1 == False 
                        and WriteToPropogator0 == False and WriteToPropogator1 == True and WriteToOut == False):  
                    
                    SSSDSYNHIGH  = 0b00100000000000010010000000000000
                    
                elif(ReadFromMain == False and ReadFromPropogator0 == False and ReadFromPropogator1 == True 
                        and WriteToPropogator0 == True and WriteToPropogator1 == False and WriteToOut == False): 
                    
                    SSSDSYNHIGH  = 0b00100000000011000000000000000000
    
                sssdsnyhighmask = (1 << self.ParameterMemoryAddressWidth) - 1  
                parameter_bits = NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseHighAddress & sssdsnyhighmask  
    
                SSSDSYNHIGH |= parameter_bits  
                print(f"SSSDSYNHIGH Command = {bin(SSSDSYNHIGH)}")
                print(f"Writing SSSDSYNHIGH Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                self.WriteNeuronParameter(column,self.LastParameterAddress[column],SSSDSYNHIGH)
                print(f"Reading back SSSDSYNHIGH Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                SSSDSYNHIGHBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                
                if SSSDSYNHIGHBACK == SSSDSYNHIGH:
                    print("Verified. Proceeding next.")
                else:
                    print("WRITE ERROR")
                
                self.LastParameterAddress[column] = self.LastParameterAddress[column]+1
                
##########################################################  RFPLST Command Generation #####################################

                RFPLST  = 0b00110000000000000000000011111111
                
                print(f"RFPLST Command = {bin(RFPLST)}")
                print(f"Writing RFPLST Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                self.WriteNeuronParameter(column,self.LastParameterAddress[column],RFPLST)
                print(f"Reading back RFPLST Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                RFPLSTBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                                
                if RFPLSTBACK == RFPLST:
                    print("Verified. Proceeding next.")
                else:
                    print("WRITE ERROR")
                
                self.LastParameterAddress[column] = self.LastParameterAddress[column]+1
                
############################################## PFLOWSYNQ Command Generation #####################################
                
                PFLOWSYNQ = 0b01000000000000000000000000000000
                
                synlowmask = 0xFFFF

                synq = (NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapticQFactor) & synlowmask
               
                PFLOWSYNQ |= synq
                
                pflowshift = (NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].ProgramFlowLowAddress << 16)

                pflowmask = 0x3FF0000

                pflow = pflowshift & pflowmask

                PFLOWSYNQ |= pflow
             
                print(f"PFLOWSYNQ Command = {bin(PFLOWSYNQ)}")
                print(f"Writing PFLOWSYNQ Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                self.WriteNeuronParameter(column,self.LastParameterAddress[column],PFLOWSYNQ)
                print(f"Reading back PFLOWSYNQ Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                PFLOWSYNQBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                                
                if PFLOWSYNQBACK == PFLOWSYNQ:
                    print("Verified. Proceeding next.")
                else:
                    print("WRITE ERROR")
                
                self.LastParameterAddress[column] = self.LastParameterAddress[column]+1

########################################################## ULEARNPARAMS Command Generation #############################
                                
                if NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseMode == 'Chaos': # No Prunning, Softlimits Ignored, Zeros Not Ignored
                    
                    ULEARNPARAMS = 0b01010001000000000000000000000000
                  
                elif NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseMode == 'Normal': # Optional Prunning, Softlimits Not Ignored, Zeros Ignored
                    
                    if NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].Prunning == True : 
                        ULEARNPARAMS = 0b01010110000000000000000000000000
                    else:
                        ULEARNPARAMS = 0b01010010000000000000000000000000

                wmaxmask = 0xFF0000
                wminmask = 0xFF00
                prunthreshmask = 0xFF
                
                wmax = (NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseUpperLimit << 16)
                wmin = (NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].SynapseLowerLimit << 8)

                thresh = (NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].PrunThreshold)

                wmaxwr = wmax & wmaxmask
                
                ULEARNPARAMS |= wmaxwr
                
                wminwr = wmin & wminmask
                
                ULEARNPARAMS |= wminwr

                if thresh is not None:
                    prunthreshwr = thresh & prunthreshmask
                    ULEARNPARAMS |= prunthreshwr
                             
                print(f"ULEARNPARAMS Command = {bin(ULEARNPARAMS)}")
                print(f"Writing ULEARNPARAMS Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                self.WriteNeuronParameter(column,self.LastParameterAddress[column],ULEARNPARAMS)
                print(f"Reading back ULEARNPARAMS Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                ULEARNPARAMSBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                                
                if ULEARNPARAMSBACK == ULEARNPARAMS:
                    print("Verified. Proceeding next.")
                else:
                    print("WRITE ERROR")
                
                self.LastParameterAddress[column] = self.LastParameterAddress[column]+1
                
########################################################## NPADDRDATA Command Generation #############################

                addrmask = 0x3FF0000
                datamask = 0xFFFF
                
                if isinstance(NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].NeuronParamAddrArray, list):
                    for i in range(len(NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].NeuronParamAddrArray)):
                        NPADDRDATA = 0b01100000000000000000000000000000
                        addr=(NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].NeuronParamAddrArray[i]<<16)
                        data=(NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].NeuronParamDataArray[i])
                        
                        addrwr = addr & addrmask
                        datawr = data & datamask
                        NPADDRDATA |= addrwr
                        NPADDRDATA |= datawr
                        
                        print(f"NPADDRDATA Command = {bin(NPADDRDATA)}")
                        print(f"Writing NPADDRDATA Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                        self.WriteNeuronParameter(column,self.LastParameterAddress[column],NPADDRDATA)
                        print(f"Reading back NPADDRDATA Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                        NPADDRDATASBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                                
                        if NPADDRDATABACK == NPADDRDATA:
                            print("Verified. Proceeding next.")
                        else:
                            print("WRITE ERROR")
                
                        self.LastParameterAddress[column] = self.LastParameterAddress[column]+1
                else:
                    NPADDRDATA = 0b01100000000000000000000000000000
                    addr=(NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].NeuronParamAddrArray<<16)
                    data=(NeuralNet.neurons[NeuronInProcess + NeuronIndexOffset].NeuronParamDataArray)
                        
                    addrwr = addr & addrmask
                    datawr = data & datamask
                    NPADDRDATA |= addrwr
                    NPADDRDATA |= datawr
                        
                    print(f"NPADDRDATA Command = {bin(NPADDRDATA)}")
                    print(f"Writing NPADDRDATA Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                    self.WriteNeuronParameter(column,self.LastParameterAddress[column],NPADDRDATA)
                    print(f"Reading back NPADDRDATA Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                    NPADDRDATABACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                                
                    if NPADDRDATABACK == NPADDRDATA:
                        print("Verified. Proceeding next.")
                    else:
                        print("WRITE ERROR")
                
                    self.LastParameterAddress[column] = self.LastParameterAddress[column]+1                    
                

########################################################## ENDFLOW Command Generation #############################

                if LayerIsOutput == True:
                    ENDFLOW = 0b01110000000000000000000000000010
            
                    print(f"ENDFLOW Command = {bin(ENDFLOW)}")
                    print(f"Writing ENDFLOW Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                    self.WriteNeuronParameter(column,self.LastParameterAddress[column],ENDFLOW)
                    print(f"Reading back NPADDRDATA Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                    ENDFLOWBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                                
                    if ENDFLOWBACK == ENDFLOW:
                        print("Verified. Proceeding next.")
                    else:
                        print("WRITE ERROR")
                
                    self.LastParameterAddress[column] = self.LastParameterAddress[column]+1  
            
                    ENDFLOW = 0b01110000000000000000000000000000
                
                    print(f"ENDFLOW Command = {bin(ENDFLOW)}")
                    print(f"Writing ENDFLOW Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                    self.WriteNeuronParameter(column,self.LastParameterAddress[column],ENDFLOW)
                    print(f"Reading back ENDFLOW Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                    ENDFLOWBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                                
                    if ENDFLOWBACK == ENDFLOW:
                        print("Verified. Proceeding next.")
                    else:
                        print("WRITE ERROR")
                
                    self.LastParameterAddress[column] = self.LastParameterAddress[column]+1  
                
                else:
                    
                    ENDFLOW = 0b01110000000000000000000000000001
                    
                    print(f"ENDFLOW Command = {bin(ENDFLOW)}")
                    print(f"Writing ENDFLOW Command to Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                
                    self.WriteNeuronParameter(column,self.LastParameterAddress[column],ENDFLOW)
                    print(f"Reading back ENDFLOW Command from Column = {column} parameter memory address {self.LastParameterAddress[column]}")
                    ENDFLOWBACK=self.ReadNeuronParameter(column,self.LastParameterAddress[column])
                                
                    if ENDFLOWBACK == ENDFLOW:
                        print("Verified. Proceeding next.")
                    else:
                        print("WRITE ERROR")
                
                    self.LastParameterAddress[column] = self.LastParameterAddress[column]+1  


            NeuronIndexOffset += LayerSize
        
        print("NETWORK MAPPING COMPLETED!..")
        
##########################################################################################################################                

    def StartNewTimestep(self):
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS);
        mask = (1 << 9)
        ControlRegTemp |= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        mask = ~(1 << 9)
        ControlRegTemp &= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        
    def ReadNeuronSynapse(self,Column,SynapseAddr):
        
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS);
        mask = (1 << 3)
        ControlRegTemp |= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        
        SelectColumn = int(Column)
        Synapses.write(SYNAPSE_MEM_SELECT,SelectColumn)
        BitwiseColumn = 0
        BitwiseColumn |= (1 << SelectColumn)
        Synapses.write(SYNAPSE_MEM_EN,BitwiseColumn)
        Synapses.write(SYNAPSE_MEM_ADDR,SynapseAddr)
        SynapseValue = Synapses.read(SYNAPSE_MEM_DOUT)
        Synapses.write(SYNAPSE_MEM_EN,0)
        
        mask = ~(1 << 3)
        ControlRegTemp &= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        
        return SynapseValue

    def ReadNeuronParameter(self,Column,ParamAddr):
        
        SelectColumn = int(Column)
        Parameter.write(PARAMETER_MEM_SELECT,SelectColumn)
        BitwiseColumn = 0
        BitwiseColumn |= (1 << SelectColumn)
        Parameter.write(PARAMETER_MEM_EN,BitwiseColumn)
        Parameter.write(PARAMETER_MEM_ADDR,ParamAddr)
        ParameterValue = Parameter.read(PARAMETER_MEM_DOUT)
        Parameter.write(PARAMETER_MEM_EN,0)
        
        return ParameterValue

    def WriteNeuronSynapse(self,Column,SynapseAddr,SynapseData):
        
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS);
        mask = (1 << 3)
        ControlRegTemp |= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
   
        SelectColumn = int(Column)
        Data = int(SynapseData)
        Synapses.write(SYNAPSE_MEM_SELECT,SelectColumn)
        BitwiseColumn = 0
        BitwiseColumn |= (1 << SelectColumn)
        Synapses.write(SYNAPSE_MEM_ADDR,SynapseAddr)
        Synapses.write(SYNAPSE_MEM_DIN,Data)
        Synapses.write(SYNAPSE_MEM_EN,BitwiseColumn)
        Synapses.write(SYNAPSE_MEM_WREN,BitwiseColumn)
        Synapses.write(SYNAPSE_MEM_EN,0)
        Synapses.write(SYNAPSE_MEM_WREN,0)
        self.FreeSynapticMemory[Column] = self.FreeSynapticMemory[Column] -1
        mask = ~(1 << 3)
        ControlRegTemp &= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        
        self.LastSynapseAddress[Column]= SynapseAddr

    def WriteNeuronParameter(self,Column,ParameterAddr,ParameterData):
        SelectColumn = int(Column)
        Data = int(ParameterData)
        Parameter.write(PARAMETER_MEM_SELECT,SelectColumn)
        BitwiseColumn = 0
        BitwiseColumn |= (1 << SelectColumn)
        Parameter.write(PARAMETER_MEM_ADDR,ParameterAddr)
        Parameter.write(PARAMETER_MEM_DIN,Data)
        Parameter.write(PARAMETER_MEM_EN,BitwiseColumn)
        Parameter.write(PARAMETER_MEM_WREN,BitwiseColumn)
        Parameter.write(PARAMETER_MEM_EN,0)
        Parameter.write(PARAMETER_MEM_WREN,0)
        self.FreeParameterMemory[Column] = self.FreeParameterMemory[Column] -1
        self.LastParameterAddress[Column]=ParameterAddr
        
    def Set(self):
        
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS)
        print(bin(ControlRegTemp))
        mask = (1 << 10) | (1 << 11) | (1 << 12)
        ControlRegTemp |= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS)
        print(bin(ControlRegTemp))
        mask = ~((1 << 10) | (1 << 11) | (1 << 12))
        ControlRegTemp &= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS)
        print(bin(ControlRegTemp))
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS);
        mask = (1 << 9) 
        ControlRegTemp |= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS)
        print(bin(ControlRegTemp))
        mask = ~(1 << 9) 
        ControlRegTemp &= mask
        Controls.write(PROCESSOR_CONTROLS,ControlRegTemp)
        ControlRegTemp = Controls.read(PROCESSOR_CONTROLS)
        print(bin(ControlRegTemp))
        
    def GetHardwareStatus(self):
        
        Status = Controls.read(GLOBAL_TIMESTEP_UPDATE_REQUEST)
        
        return Status
    
    def PushSpikesToHardware(self,InSpikeVectorCount):
        pass
    
    def PullSpikesFromHardware(self,OutSpikeVectorCount):
        pass
    
        

In [36]:
SNN_PROCESSOR = Hardware()
SNN_PROCESSOR.Generate()
SNN_PROCESSOR.GetHardwareInfo()
SNN_PROCESSOR.ShowFreeMemory()

Crossbar dimensions 16x16.
16 Neuron Model Calculator (NMC) units.
16 Learning Engines.
Maximum 4096 neurons.
Maximum 131072 synapses.
Column 0: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 1: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 2: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 3: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 4: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 5: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 6: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 7: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 8: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 9: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 10: Free Synaptic Memory 8192 Words Free Parameter Memory 2048 Words
Column 11: Free Synaptic Memory 8192 Wo

In [37]:
class Neuron:
    def __init__(self,NeuronID,ProgramFlowLowAddress,LastSpikeTime,SynapticQFactor,NeuronParamDataArray
                 ,NeuronParamAddrArray,NeuronRefractoryDuration,Synapses,SynapseUpperLimit,SynapseLowerLimit
                 ,SynapseMode,Prunning,PrunThreshold,SynapseLowAddress,SynapseHighAddress,Position):

        self.NeuronID = NeuronID
        self.ProgramFlowLowAddress = ProgramFlowLowAddress
        self.LastSpikeTime = LastSpikeTime
        self.SynapticQFactor = SynapticQFactor
        self.NeuronParamDataArray = NeuronParamDataArray
        self.NeuronParamAddrArray = NeuronParamAddrArray
        self.NeuronRefractoryDuration = NeuronRefractoryDuration
        self.Synapses = Synapses
        self.SynapseUpperLimit = SynapseUpperLimit
        self.SynapseLowerLimit = SynapseLowerLimit
        self.SynapseMode = SynapseMode
        self.Prunning = Prunning
        self.PrunThreshold = PrunThreshold
        self.SynapseLowAddress = SynapseLowAddress
        self.SynapseHighAddress = SynapseHighAddress
        self.Position = Position

 

In [38]:
class Network():
    def __init__(self, input_size):
        self.input_size = input_size
        self.LayerSizes = []
        self.NeuronCount = 0
        self.LayerSizes.append(input_size)
        self.neurons = []  
        self.ParameterSpace = []
        self.previous_neuron_index = 0
        
    def Append(self, NMODELLowAddr, ParamData, ParamAddr, SynapseType, Weights, LayerSize, Scheme, Probability,
           Wmax, Wmin, LearningMode, WeightPrunning, PrunningThreshold):
        if self.NeuronCount + LayerSize > SNN_PROCESSOR.NeuronCapacity:
            raise ValueError(f"Network neuron limit {SNN_PROCESSOR.NeuronCapacity} is exceeded.")
        if LayerSize > SNN_PROCESSOR.SynapticMemoryPerColumn:
            raise ValueError(f"Layer size cannot exceed {SNN_PROCESSOR.SynapticMemoryPerColumn}.")                
        if Scheme not in ['fc', 'random']:
            raise ValueError("Unrecognized connection scheme.") 
        if LearningMode not in ['Chaos', 'Normal']:
            raise ValueError("Unrecognized learning mode.") 
    
        self.NeuronCount += LayerSize
        self.LayerSizes.append(LayerSize)

        if abs(Wmax) >= abs(Wmin):
            LayerUpperWeightLimit = +abs(Wmax) 
            LayerLowerWeightLimit = -abs(Wmax)  
        else:
            LayerUpperWeightLimit = +abs(Wmin)  
            LayerLowerWeightLimit = -abs(Wmin) 
            
        QFACTOR =  (LayerUpperWeightLimit - LayerLowerWeightLimit)/255
        
        SynQFactor = HalfPrecision2Bin(QFACTOR)
        
        NeuronWMax = (round(Wmax / QFACTOR))
        NeuronWmin = (round(Wmin / QFACTOR))
        
        if PrunningThreshold is not None:
            Pthresh    = round(PrunningThreshold / QFACTOR)
            PT = int(Pthresh)
        else:
            PT = None  
            
        if(NeuronWMax == 128):
            NeuronWMax = 127
        
        NeuronWMax = signed_to_uint8(NeuronWMax)
        NeuronWmin = signed_to_uint8(NeuronWmin)
            
        neurons = []
        for NeuronIndex in range(1, LayerSize + 1):
            updated_neuron_index = self.previous_neuron_index + NeuronIndex
        
            neuron = Neuron(updated_neuron_index, ProgramFlowLowAddress=NMODELLowAddr, LastSpikeTime=0x7F,
                        SynapticQFactor=SynQFactor, NeuronParamDataArray=ParamData,
                        NeuronParamAddrArray=ParamAddr, NeuronRefractoryDuration=0,
                        Synapses=None, SynapseUpperLimit=int((NeuronWMax)), 
                        SynapseLowerLimit=int((NeuronWmin)),
                        SynapseMode=LearningMode, Prunning=WeightPrunning, PrunThreshold=PT
                        ,SynapseLowAddress=None,SynapseHighAddress=None,Position=None)
        
            if Scheme == 'fc':
                if isinstance(Weights, str):
                    if Weights == 'r':
                        synapsearray = CreateRandomSynapse(self.LayerSizes[-2])
                    else:
                        raise ValueError("Unrecognized weight definition. For random, use 'r', for a particular number between -1 and 1, give a number.")          
                elif isinstance(Weights, numbers.Number):
                    SynapticQfactor = 2/255
                    StartVal = round(Weights / SynapticQfactor)
                    StaWeightsrtVal = signed_to_uint8(StartVal)
                    synapsearray = np.full(self.LayerSizes[-2], StartVal, dtype=np.uint8)
                else:
                    raise ValueError("Unrecognized weight definition. For random, use 'r', for a particular number between -1 and 1, give a number.")          
                            
                synaptic_data = InitializeTimestamps(synapsearray)
                neuron.Synapses = synaptic_data
            
            elif Scheme == 'random':
                synapsearray = CreateRandomSynapse(self.LayerSizes[-2])
                synaptic_data = InitializeTimestamps(synapsearray)
                
                for SynapseIndex in range(self.LayerSizes[-2]):
                    connect_probability = random.random() 

                    if Probability >= connect_probability:
                        synaptic_data[SynapseIndex] = synaptic_data[SynapseIndex]
                    else:
                        synaptic_data[SynapseIndex] = 0   
                    
                neuron.Synapses = synaptic_data
                
            neurons.append(neuron)
    
    # Update the previous_neuron_index after processing the entire layer
        self.previous_neuron_index += LayerSize
    
        self.neurons.extend(neurons)


In [39]:
# Creating LearningWindow Input Arguments

TimewindowLeft = 50e-3
TimewindowRight = 50e-3
ZeroPoint= 0
Samples= 256
on_pre = '''0.6*exp(t/10e-3)'''
on_post = '''-0.6*exp(-t/10e-3)'''

BiphasicSTDP = Synapse(TimewindowLeft, TimewindowRight, ZeroPoint, Samples, on_pre, on_post)

In [40]:
# Spike Response Model Assembly Code
# Address of Parameter 3C00 (FP16 1.0) = 59
# Address of Parameter Pmin  = 60
# Address of Parameter Pth   = 61
# Address of Parameter D     = 62
# Address of Parameter Pref  = 63
# Address of Parameter Rp    = 64
# Address of Parameter Pt-1 (Membran Voltage/State Variable) = 65
SRM = '''
lw,x0,59
lw,x2,60
lw,x3,61
lw,x4,62
lw,x5,63
lw,x6,64
lw,x7,65
comp,x7,x2
big,3
sw,x6,65
return
comp,x7,x3
bil,5
sw,x5,65
spk
strf,27
return
fmac,x7,x0
smac,x4,x0
getacc,x7
clracc
sw,x7,65
return
'''

In [41]:
#    Leaky Integrate and Fire Neuron (Without Refractory, Constant Input Current)
#    
#    Kernel -->   V' = (R*I-V)/t_m
#
#    Initial Membrane Voltage --> V       = 70 mV
#    Membrane Resistance      --> R       = 1 mOhm
#    Membrane Time Constant   --> t_m     = 10 ms
#    Threshold Voltage        --> V_th    = 20 mV
#    Reset Voltage            --> V_reset = 60 mV
#    Pre Synaptic Current     --> I       = 0.1 uA
#    Timestep (h)             --> h       = 0.01
#     
#    Discrete Time Kernel (method = Euler)      
#
#    v[n+1] = R*I*h*r_t_m - V[n]*r_t_m*h + v[n]
#
#    LiF Assembly Code (Without Pre-Processing)
LiF = '''
getacc,x1    
clracc
lw,x2,44     
lw,x3,45     
lw,x4,46     
lw,x5,57     
fmac,x5,x4   
getacc,x7    
clracc
fmac x7,x2   
smac x2,x0   
getacc,x6    
clracc
fmac x7,x3   
getacc,x7    
clracc      
fmac x7,x1   
smac x0,x6   
getacc,x7    
lw,x4,66     
comp,x7,x4   
bil,3
sw x7,44     
return
spk
lw x7,79   
sw x7,44
return
'''

In [42]:
# Izhikevich Neuron Model
IZH = '''
getacc,x1  
clracc     
lw,x2,44   
lw,x3,45   
lw,x4,46   
lw,x5,47   
fmac,x2,x0 
fmac,x1,x3 
smac,x4,x3 
fmac,x5,x3 
getacc,x6  
clracc
fmac,x3,x2 
getacc,x7  
clracc
lw,x5,48   
fmac,x7,x5 
fmac,x6,x0 
getacc,x6  
clracc
lw,x5,49   
fmac,x7,x5 
getacc,x7  
clracc
fmac,x7,x2 
fmac,x6,x0 
getacc,x6  
clracc
lw,x7,53   
comp,x6,x7 
big,26
sw,x6,44   
lw,x5,51   
lw,x6,52   
fmac,x5,x6 
getacc,x7  
clracc
fmac,x2,x3 
getacc,x1  
clracc
fmac,x1,x7 
fmac,x4,x0 
getacc,x1  
clracc
fmac,x3,x5 
getacc,x7  
clracc
fmac,x7,x4 
getacc,x7  
clracc
fmac,x0,x1 
smac,x0,x7 
getacc,x7  
clracc
sw,x7,46   
return
spk
lw,x7,50   
sw,x7,44   
fmac,x0,x4 
lw,x2,54   
fmac,x2,x0 
getacc,x7  
sw,x7,46   
return
'''

In [43]:
# NMC Neuron Parameter Pass
# NPARAM_DATA[31:16] & NPARAM_ADDR[15:0]

# Address of Parameter 3C00 (FP16 1.0) = 59
# Address of Parameter Pmin  = 60
# Address of Parameter Pth   = 61
# Address of Parameter D     = 62
# Address of Parameter Pref  = 63
# Address of Parameter Rp    = 64
# Address of Parameter Pt-1 (Membran Voltage/State Variable) = 65

# Value of Parameter Pmin      = -37 (D0A0)
# Value of Parameter Pth       = 77  (54D0)
# Value of Parameter D         = 3   (4200)
# Value of Parameter Prefract  = 2   (4000)
# Value of Parameter Rp        = 0   (0)
# Value of Parameter Pt-1 (Membran Voltage/State Variable) = 65

SRM_Baseaddr = int(0)

SRMProgFlow = NModelAssembler(SRM,SRM_Baseaddr)

SNN_PROCESSOR.LoadNeuronKernel(SRMProgFlow)

SRMProgFlowLow = SRM_Baseaddr
SRMParamData = int(0)
SRMParamAddr = int(65)

SNN_PROCESSOR.LoadNeuronParameters(SRMParamData,SRMParamAddr)
SNN_PROCESSOR.LoadNeuronParameters(0x3C00,0x003B)
SNN_PROCESSOR.LoadNeuronParameters(0xD0A0,0x003C)
SNN_PROCESSOR.LoadNeuronParameters(0x54D0,0x003D)
SNN_PROCESSOR.LoadNeuronParameters(0x4200,0x003E)
SNN_PROCESSOR.LoadNeuronParameters(0x4000,0x003F)
SNN_PROCESSOR.LoadNeuronParameters(0x0000,0x0040)
SNN_PROCESSOR.LoadNeuronParameters(0x0000,0x0041)
SNN_PROCESSOR.LoadLearningTable(BiphasicSTDP)
        
   

In [44]:
# ONLY FOR TEST

file_name = './semeion/semeion_streamdata_full_10.dat'

In [45]:
# ONLY FOR TEST

SNN = Network(input_size=768)
SNN.Append(NMODELLowAddr=SRMProgFlowLow, ParamData=SRMParamData, ParamAddr=SRMParamAddr, 
           SynapseType=BiphasicSTDP, Weights='r', LayerSize = 128, Scheme='fc', Probability=None,
           Wmax=1,Wmin=-1,LearningMode='Normal',WeightPrunning=None,PrunningThreshold=None)
SNN.Append(NMODELLowAddr=SRMProgFlowLow, ParamData=SRMParamData, ParamAddr=SRMParamAddr, 
           SynapseType=BiphasicSTDP, Weights='r', LayerSize = 128, Scheme='fc', Probability=None,
           Wmax=1,Wmin=-1,LearningMode='Normal',WeightPrunning=None,PrunningThreshold=None)

In [46]:
# ONLY FOR TEST

SNN_PROCESSOR.MapNetwork(SNN)

Mapping Input Layer with 128 neurons:
Processing Neuron ID = 1
Number of Neuron Synapses = 768
Neuron ID 1 is being written to column 0
Neuron Synapse Low Address = 0
Neuron Synapse High Address = 767
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000000000000000000
Writing SYNLOW Command to Column = 0 parameter memory address 0
Reading back SYNLOW Command from Column = 0 parameter memory address 0
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000010010000001011111111
Writing SSSDSYNHIGH Command to Column = 0 parameter memory address 1
Reading back SSSDSYNHIGH Command from Column = 0 parameter memory address 1
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 0 parameter memory address 2
Reading back RFPLST Command from Column = 0 parameter memory address 2
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 0

Neuron Synapse Low Address = 0
Neuron Synapse High Address = 767
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000000000000000000
Writing SYNLOW Command to Column = 5 parameter memory address 0
Reading back SYNLOW Command from Column = 5 parameter memory address 0
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000010010000001011111111
Writing SSSDSYNHIGH Command to Column = 5 parameter memory address 1
Reading back SSSDSYNHIGH Command from Column = 5 parameter memory address 1
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 5 parameter memory address 2
Reading back RFPLST Command from Column = 5 parameter memory address 2
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 5 parameter memory address 3
Reading back PFLOWSYNQ Command from Column = 5 parameter memory address 3
Verified. Proceeding next.
ULEARNP

Neuron Synapse Low Address = 0
Neuron Synapse High Address = 767
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000000000000000000
Writing SYNLOW Command to Column = 10 parameter memory address 0
Reading back SYNLOW Command from Column = 10 parameter memory address 0
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000010010000001011111111
Writing SSSDSYNHIGH Command to Column = 10 parameter memory address 1
Reading back SSSDSYNHIGH Command from Column = 10 parameter memory address 1
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 10 parameter memory address 2
Reading back RFPLST Command from Column = 10 parameter memory address 2
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 10 parameter memory address 3
Reading back PFLOWSYNQ Command from Column = 10 parameter memory address 3
Verified. Proceeding next.

Neuron Synapse Low Address = 768
Neuron Synapse High Address = 1535
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000000001100000000
Writing SYNLOW Command to Column = 0 parameter memory address 7
Reading back SYNLOW Command from Column = 0 parameter memory address 7
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000010010000010111111111
Writing SSSDSYNHIGH Command to Column = 0 parameter memory address 8
Reading back SSSDSYNHIGH Command from Column = 0 parameter memory address 8
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 0 parameter memory address 9
Reading back RFPLST Command from Column = 0 parameter memory address 9
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 0 parameter memory address 10
Reading back PFLOWSYNQ Command from Column = 0 parameter memory address 10
Verified. Proceeding next.
UL

Neuron Synapse Low Address = 768
Neuron Synapse High Address = 1535
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000000001100000000
Writing SYNLOW Command to Column = 5 parameter memory address 7
Reading back SYNLOW Command from Column = 5 parameter memory address 7
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000010010000010111111111
Writing SSSDSYNHIGH Command to Column = 5 parameter memory address 8
Reading back SSSDSYNHIGH Command from Column = 5 parameter memory address 8
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 5 parameter memory address 9
Reading back RFPLST Command from Column = 5 parameter memory address 9
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 5 parameter memory address 10
Reading back PFLOWSYNQ Command from Column = 5 parameter memory address 10
Verified. Proceeding next.
UL

...

Neuron Synapse Low Address = 6912
Neuron Synapse High Address = 7039
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000001101100000000
Writing SYNLOW Command to Column = 5 parameter memory address 104
Reading back SYNLOW Command from Column = 5 parameter memory address 104
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000100100000001101111111
Writing SSSDSYNHIGH Command to Column = 5 parameter memory address 105
Reading back SSSDSYNHIGH Command from Column = 5 parameter memory address 105
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 5 parameter memory address 106
Reading back RFPLST Command from Column = 5 parameter memory address 106
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 5 parameter memory address 107
Reading back PFLOWSYNQ Command from Column = 5 parameter memory address 107
Verified. Proc

Neuron Synapse Low Address = 6912
Neuron Synapse High Address = 7039
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000001101100000000
Writing SYNLOW Command to Column = 11 parameter memory address 104
Reading back SYNLOW Command from Column = 11 parameter memory address 104
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000100100000001101111111
Writing SSSDSYNHIGH Command to Column = 11 parameter memory address 105
Reading back SSSDSYNHIGH Command from Column = 11 parameter memory address 105
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 11 parameter memory address 106
Reading back RFPLST Command from Column = 11 parameter memory address 106
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 11 parameter memory address 107
Reading back PFLOWSYNQ Command from Column = 11 parameter memory address 107
Verifi

Neuron Synapse Low Address = 7040
Neuron Synapse High Address = 7167
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000001101110000000
Writing SYNLOW Command to Column = 4 parameter memory address 112
Reading back SYNLOW Command from Column = 4 parameter memory address 112
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000100100000001111111111
Writing SSSDSYNHIGH Command to Column = 4 parameter memory address 113
Reading back SSSDSYNHIGH Command from Column = 4 parameter memory address 113
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 4 parameter memory address 114
Reading back RFPLST Command from Column = 4 parameter memory address 114
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 4 parameter memory address 115
Reading back PFLOWSYNQ Command from Column = 4 parameter memory address 115
Verified. Proc

Neuron Synapse Low Address = 7040
Neuron Synapse High Address = 7167
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000001101110000000
Writing SYNLOW Command to Column = 10 parameter memory address 112
Reading back SYNLOW Command from Column = 10 parameter memory address 112
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000100100000001111111111
Writing SSSDSYNHIGH Command to Column = 10 parameter memory address 113
Reading back SSSDSYNHIGH Command from Column = 10 parameter memory address 113
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 10 parameter memory address 114
Reading back RFPLST Command from Column = 10 parameter memory address 114
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 10 parameter memory address 115
Reading back PFLOWSYNQ Command from Column = 10 parameter memory address 115
Verifi

Neuron Synapse Low Address = 7040
Neuron Synapse High Address = 7167
Compiling Bridge Instructions for Neuron ID {neuron_id}
SYNLOW Command = 0b10000000000000001101110000000
Writing SYNLOW Command to Column = 15 parameter memory address 112
Reading back SYNLOW Command from Column = 15 parameter memory address 112
Verified. Proceeding next.
SSSDSYNHIGH Command = 0b100000000100100000001111111111
Writing SSSDSYNHIGH Command to Column = 15 parameter memory address 113
Reading back SSSDSYNHIGH Command from Column = 15 parameter memory address 113
Verified. Proceeding next.
RFPLST Command = 0b110000000000000000000011111111
Writing RFPLST Command to Column = 15 parameter memory address 114
Reading back RFPLST Command from Column = 15 parameter memory address 114
Verified. Proceeding next.
PFLOWSYNQ Command = 0b1000000000000000010000000000100
Writing PFLOWSYNQ Command to Column = 15 parameter memory address 115
Reading back PFLOWSYNQ Command from Column = 15 parameter memory address 115
Verifi

In [52]:
# ONLY FOR TEST

SNN_PROCESSOR.Set()

0b0
0b1110000000000
0b0
0b1000000000
0b0


In [53]:
# ONLY FOR TEST
SpikeFeed = np.random.randint(0, 2**16, size=960, dtype=np.uint32)
SpikeBuffer = allocate(shape=(len(SpikeFeed),), dtype=np.uint32)

for l in range(len(SpikeFeed)):
    SpikeBuffer[l] = SpikeFeed[l]
    #print(hex(SpikeBuffer[l]))
PushSpikes.transfer(SpikeBuffer)
PushSpikes.wait()

In [54]:
# ONLY FOR TEST

Stats= SNN_PROCESSOR.GetHardwareStatus()

In [55]:
# ONLY FOR TEST

print(bin(Stats))

0b0


In [51]:
OutSpikeBuffer = allocate(96,), dtype=np.uint32)

PullSpikes.transfer(OutSpikeBuffer)
PullSpikes.wait()

SyntaxError: invalid syntax (<ipython-input-51-0ac97c7524a1>, line 1)